In [40]:
addresses = [
    "King's Parade, Cambridge CB2 1SJ, UK",
    "Trinity College, Cambridge CB2 1TQ, UK",
    "24 Hills Road, Cambridge CB2 0QQ, UK",
    "St John's College, St John's St, Cambridge CB2 1TP, UK",
    "Parker's Piece, Cambridge CB1 1JF, UK",
    "The Clarendon Arms, 35 Clarendon Street, Cambridge CB1 1JX, UK",
    "5 Sidgwick Avenue, Cambridge CB3 9DA, UK",
    "10 West Road, Cambridge CB3 9DZ, UK",
    "Cambridge Science Centre, 44 Clifton Rd, Cambridge CB1 7ED, UK"]

In [41]:
import osmnx as ox
import networkx as nx
import folium
from geopy.geocoders import Nominatim
from geopy.distance import great_circle
import itertools




geolocator = Nominatim(user_agent="myGeocoder")
coords_list = [geolocator.geocode(address).point[:2] for address in addresses]

# Get the street network of Cambridge
place_name = "Cambridge, UK"
G = ox.graph_from_place(place_name, network_type='walk')

/Users/paulcarroll/Github/nhs_time_of_travel/geospat/lib/python3.10/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


In [42]:
locations = []
for address in addresses:
    location = geolocator.geocode(address)
    if location is not None:
        locations.append((location.latitude, location.longitude))
    else:
        print(f"Could not find location for address: {address}")



In [43]:
import itertools
'''
def distance(city1, city2):
    # Calculates the Euclidean distance between two cities
    x1, y1 = city1
    x2, y2 = city2
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5
'''

def manhattan_distance(city1, city2):
    # Calculates the Manhattan distance between two cities
    x1, y1 = city1
    x2, y2 = city2
    return abs(x1 - x2) + abs(y1 - y2)


def tsp(cities, addresses):
    # Calculates the shortest possible route that visits all cities and returns to the starting city
    shortest_distance = float('inf')
    shortest_route = None
    city_to_address = dict(zip(cities, addresses))
    for route in itertools.permutations(cities):
        route_distance = 0
        for i in range(len(route)):
            if i == len(route) - 1:
                route_distance += manhattan_distance(route[i], route[0])
            else:
                route_distance += manhattan_distance(route[i], route[i+1])
        if route_distance < shortest_distance:
            shortest_distance = route_distance
            shortest_route = route
    shortest_route_addresses = [city_to_address[city] for city in shortest_route]
    return shortest_route_addresses, shortest_distance



In [44]:
shortest_route_addresses, shortest_distance = tsp(locations, addresses)
print(f"Shortest route: {shortest_route_addresses}")
print(f"Shortest distance: {shortest_distance}")

Shortest route: ["King's Parade, Cambridge CB2 1SJ, UK", 'Trinity College, Cambridge CB2 1TQ, UK', "St John's College, St John's St, Cambridge CB2 1TP, UK", '10 West Road, Cambridge CB3 9DZ, UK', '5 Sidgwick Avenue, Cambridge CB3 9DA, UK', '24 Hills Road, Cambridge CB2 0QQ, UK', 'Cambridge Science Centre, 44 Clifton Rd, Cambridge CB1 7ED, UK', "Parker's Piece, Cambridge CB1 1JF, UK", 'The Clarendon Arms, 35 Clarendon Street, Cambridge CB1 1JX, UK']
Shortest distance: 0.09326910000001015


In [61]:
import folium

# Create a map centered on the first address in the shortest route
map_centre = locations[0]
m = folium.Map(location=map_centre, zoom_start=13)


for i, address in enumerate(shortest_route_addresses):
    location = geolocator.geocode(address)
    if location is not None:
        lat, lon = location.latitude, location.longitude
        tooltip = f"{i+1}. {address}"
        folium.Marker(location=[lat, lon], tooltip=tooltip).add_to(m)
    else:
        print(f"Could not find location for address: {address}")

locations_ordered = []
for address in shortest_route_addresses:
    location = geolocator.geocode(address)
    if location is not None:
        lat, lon = location.latitude, location.longitude
        locations_ordered.append((lat, lon))
    else:
        print(f"Could not find location for address: {address}")

nodes = [ox.distance.nearest_nodes(G, X=[coord[1]], Y=[coord[0]])[0] for coord in locations_ordered]
shortest_paths = [nx.shortest_path(G, nodes[i], nodes[i + 1], weight='length') for i in range(len(nodes) - 1)]
path_lengths = [nx.shortest_path_length(G, nodes[i], nodes[i + 1], weight='length')/1609.34 for i in range(len(nodes) - 1)]


for path in shortest_paths:
    route_nodes_coords = [G.nodes[node] for node in path]
    route_coords = [(node['y'], node['x']) for node in route_nodes_coords]
    folium.PolyLine(route_coords, color='blue', weight=2.5).add_to(m)

In [49]:
locations_ordered

[(52.2044024, 0.1176368),
 (52.20689025, 0.11511292588941303),
 (52.20875705, 0.11349862269053744),
 (52.2018283, 0.10966392150201955),
 (52.2004011, 0.1091291),
 (52.1928125, 0.13244875),
 (52.1927509, 0.1395232),
 (52.2019546, 0.12795768334348306),
 (52.2046367, 0.1275966)]

In [50]:
m

In [68]:

distance_data = {
    'From': shortest_route_addresses[:-1],
    'To': shortest_route_addresses[1:],
    'Distance (miles)': [round(dist,2) for dist in path_lengths],
    'Cumulative Distance (miles)': [round(sum(path_lengths[:i+1]),2) for i in range(len(path_lengths))]
}
df = pd.DataFrame(distance_data)

In [69]:
df

,From,To,Distance (miles),Cumulative Distance (miles)
0,"King's Parade, Cambridge CB2 1SJ, UK","Trinity College, Cambridge CB2 1TQ, UK",0.89,0.89
1,"Trinity College, Cambridge CB2 1TQ, UK","St John's College, St John's St, Cambridge CB2...",0.39,1.28
2,"St John's College, St John's St, Cambridge CB2...","10 West Road, Cambridge CB3 9DZ, UK",0.57,1.85
3,"10 West Road, Cambridge CB3 9DZ, UK","5 Sidgwick Avenue, Cambridge CB3 9DA, UK",0.50,2.35
4,"5 Sidgwick Avenue, Cambridge CB3 9DA, UK","24 Hills Road, Cambridge CB2 0QQ, UK",1.46,3.81
5,"24 Hills Road, Cambridge CB2 0QQ, UK","Cambridge Science Centre, 44 Clifton Rd, Cambr...",0.50,4.31
6,"Cambridge Science Centre, 44 Clifton Rd, Cambr...","Parker's Piece, Cambridge CB1 1JF, UK",1.24,5.55
7,"Parker's Piece, Cambridge CB1 1JF, UK","The Clarendon Arms, 35 Clarendon Street, Cambr...",0.33,5.87
